In [1]:
%load_ext autoreload
%autoreload 2

# Simple RAG (Retrieval-Augmented Generation) System


In [2]:
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import faiss
import os
import sys
from dotenv import load_dotenv

In [3]:
EMBED_DIMENSION = 512

# Chunk settings are way different than langchain examples
# Beacuse for the chunk length langchain uses length of the string,
# while llamaindex uses length of the tokens
CHUNK_SIZE = 10000
CHUNK_OVERLAP = 9800

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [6]:
Settings.embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={"truncate_dim": EMBED_DIMENSION})

/home/stackops/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read Docs


In [7]:
path = "/home/stackops/langchain-labs/data/vks/pdf/vi/"
node_parser = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = node_parser.load_data()
print(documents[0])

Doc ID: 22025cd3-a2ae-4bb9-b78d-aced24dfd9dd
Text: 1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud
Kubernetes Service (VKS) là gì? VNGCloud Kubernetes Service (hay còn
gọi là VKS) là một dịch vụ quản lý Kubernetes được cung cấp bởi
VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên
container một cách dễ dàng và hiệu quả. VKS giúp bạn tập trung vào
việc phát triển...


In [8]:
faiss_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=faiss_index)

## Text Cleaner Transformation


In [9]:
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        return nodes

## Ingestion Pipeline


In [10]:
text_splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

# Create a pipeline with defined document transformations and vectorstore
pipeline = IngestionPipeline(
    transformations=[
        TextCleaner(),
        text_splitter,
    ],
    vector_store=vector_store, 
)

In [11]:
# Run pipeline and get generated nodes from the process
nodes = pipeline.run(documents=documents)

## Create retriever


In [12]:
vector_store_index = VectorStoreIndex(nodes)
retriever = vector_store_index.as_retriever(similarity_top_k=5)

In [13]:
def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i+1}:")
        print(c.text)
        print("\n")

In [14]:
test_query = "So sánh các loại cluster VKS?"
context = retriever.retrieve(test_query)
show_context(context)

Context 1:
3. Mô hình hoạt động của VKS
VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private
Cluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và
Private Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để
tạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:
Public Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public
riêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này
dẫn rủi ro bảo mật cao hơn. Tuy nhiên bạn hoàn toàn có thể hạn chế truy cập đến
ControlPlane thông qua tính năng Whitelist IP. Một điều lưu ý là bạn cần đảm bảo không
hạn chế truy cập từ các IP của các node trong nodegroup.
Public Cluster với Private Node Group: Các node trong nodegroup không có địa chỉ IP
Public riêng, không thể kết nối trực tiếp đến ControlPlane. Để các node trong nodegroup
có thể kết nối đến ControlPlane, bạn cần sử dụng m

## Let's see how well does it perform:


In [ ]:
import json
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCaseParams
from evalute_rag import create_deep_eval_test_cases

/home/stackops/.local/lib/python3.10/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 2.3.7, however version 2.4.1 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


ModuleNotFoundError: No module named 'evaluation'